In [ ]:
!pip3 install -U datapane

In [ ]:
import pandas as pd
import numpy as np
import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly.io as pio
import datapane as dp

In [ ]:
symbols = ["ETH","BTC","SOL","BNB"]

In [ ]:
data_dict = {}
for sym in symbols:
    data = pd.read_csv(f"/kaggle/input/crypto-currencies-daily-prices/{sym}.csv")
    data["date"] = pd.to_datetime(data["date"], format="%Y-%m-%d")
    data_dict[sym] = data

fig_list = []
for sym, data in data_dict.items():
    data["10-day MA"] = data["close"].rolling(window=10).mean()
    data["20-day MA"] = data["close"].rolling(window=20).mean()
    trace0 = go.Scatter(x=data.date, y=data.close, name=sym)
    trace1 = go.Scatter(x=data.date, y=data["10-day MA"], name="10-day MA")
    trace2 = go.Scatter(x=data.date, y=data["20-day MA"], name="20-day MA")
    fig = go.Figure([trace0, trace1, trace2])
    fig.update_layout(title={"text": "{sym} Price", "x": 0.5, "xanchor": "center"})
    fig_list.append(fig)

In [ ]:
trace_list = []
for sym, data in data_dict.items():
    trace = go.Scatter(x=data.date, y=data.close, name=sym)
    trace_list.append(trace)
fig = go.Figure(trace_list)
fig.update_layout(
    dict(
        title=dict({"text": "TVL Top4 Price Comparision", "x": 0.5, "xanchor": "center"}),
        xaxis=dict(
            rangeselector=dict(
                buttons=list(
                    [
                        dict(count=1, label="1m", step="month", stepmode="backward"),
                        dict(count=3, label="3m", step="month", stepmode="backward"),
                        dict(count=6, label="6m", step="month", stepmode="backward"),
                        dict(count=12, label="12m", step="month", stepmode="backward"),
                    ]
                )
            ),
            rangeslider=dict(visible=True),
            type="date",
        ),
    )
)
fig_list.append(fig)


In [ ]:
trace_list = []
for sym, data in data_dict.items():
    trace = go.Scatterpolar(
        r=[data["close"].mean(), data["open"].mean(), data["low"].min(), data["high"].max()],
        theta=["Close", "Open", "Low", "High"],
        name=sym,
        fill="toself",
    )
    trace_list.append(trace)
fig = go.Figure(trace_list)
fig.update_layout(
    go.Layout(
        polar=dict(radialaxis=dict(visible=True)),
        title=dict({"text": "TVL Top4 Price Comparision (Radar Chart)", "x": 0.5, "xanchor": "center"}),
    )
)
fig_list.append(fig)

eth = data_dict["ETH"][["open", "close"]]
eth["index"] = np.arange(len(eth))
fig_eth = go.Figure(
    ff.create_scatterplotmatrix(
        eth,
        diag="box",
        index="index",
        size=3,
        height=600,
        width=1150,
        colormap="RdBu",
        title={"text": "ETH Price (Scatterplot Matrix)", "x": 0.5, "xanchor": "center"},
    )
)

In [ ]:
for fig in fig_list:
    fig.show()
fig_eth.show()

In [ ]:
def data_load() -> Dict[str, pd.DataFrame]:
    data_dict = {}
    for sym in symbols:
        data = pd.read_csv(f"./crypto_daily/{sym}.csv")
        data["date"] = pd.to_datetime(data["date"], format="%Y-%m-%d")
        data_dict[sym] = data
    return data_dict

def norm_integral(f:Callable, mean:float, std:float) -> float:
    val, er = quad(
        lambda s: np.log(1 + f * s) * norm.pdf(s, mean, std),
        mean - 3 * std,
        mean + 3 * std,
    )
    return -val

def get_kelly(data:pd.DataFrame) -> np.ndarray:
    solution = minimize_scalar(
        norm_integral, 
        args=(data["mean"], data["std"]),
        bounds=[0, 2],
        method="bounded"
    )
    return np.array(solution.x)

def gen_kelly() -> Dict[str, pd.DataFrame]:
    data_dict = data_load()
    kelly_dict = {}
    for sym, data in data_dict.items():
        X = data["close"].resample('D').last().pct_change().dropna()
        Y = X.copy()
        X = X.rolling(25).agg(["mean", "std"]).dropna
        X = X.apply(get_kelly, axis=1)
        Kelly = Y.mul(X.shift()).dropna().add(1).cumprod().sub(1)
        kelly_dict[sym] = pd.DataFrame({"date": data["date"], "close": data["close"], "kelly": Kelly}, columns=["date","close","kelly"])

    return kelly_dict

In [ ]:
kelly_dict = gen_kelly()

fig_list_k = []
for sym, data in kelly_dict.items():
    data["10-day MA"] = data["kelly"].rolling(window=10).mean()
    data["20-day MA"] = data["kelly"].rolling(window=20).mean()
    trace0 = go.Scatter(x=data.date, y=data.kelly, name=sym)
    trace1 = go.Scatter(x=data.date, y=data["10-day MA"], name="10-day MA")
    trace2 = go.Scatter(x=data.date, y=data["20-day MA"], name="20-day MA")
    fig = go.Figure([trace0, trace1, trace2])
    fig.update_layout(title={f"text": "{sym} Price", "x": 0.5, "xanchor": "center"})
    fig_list.append(fig)

trace_list = []
for sym, data in kelly_dict.items():
    trace = go.Scatter(x=data.date, y=data.kelly, name=sym)
    trace_list.append(trace)
fig = go.Figure(trace_list)
fig.update_layout(
    dict(
        title=dict({"text": "TVL Top4 Price Kelly Optd Comparision", "x": 0.5, "xanchor": "center"}),
        xaxis=dict(
            rangeselector=dict(
                buttons=list(
                    [
                        dict(count=1, label="1m", step="month", stepmode="backward"),
                        dict(count=3, label="3m", step="month", stepmode="backward"),
                        dict(count=6, label="6m", step="month", stepmode="backward"),
                        dict(count=12, label="12m", step="month", stepmode="backward"),
                    ]
                )
            ),
            rangeslider=dict(visible=True),
            type="date",
        ),
    )
)
fig_list_k.append(fig)

In [ ]:
def sharpe_ratio(returns, risk_free_rate=0.001/365, annualization_factor=365):
    if isinstance(returns, pd.Series):
        returns = returns.values 

    excess_returns = returns - risk_free_rate
    mean_excess_return = np.mean(excess_returns)
    std_dev_excess_return = np.std(excess_returns)

    if std_dev_excess_return == 0:
        return np.nan  
    else:
        sharpe = (mean_excess_return / std_dev_excess_return) * np.sqrt(annualization_factor)
        return sharpe

def get_sharpe() -> Dict[str, float]:
    data_dict = data_load()
    kelly_dict = gen_kelly()
    sharpe_dict = {}
    for sym in data_dict.keys():
        sharpe = sharpe_ratio(data_dict[sym].close)
        sharpe_kelly = sharpe_ratio(kelly_dict[sym].kelly)
        sharpe_dict[sym] = sharpe
        sharpe_dict[f"{sym}_kelly"] = sharpe_kelly
    return sharpe_dict

In [ ]:
sharpe_dict = get_sharpe()
trace_list = []
for sym, data in kelly_dict.items():
    trace = go.Scatterpolar(
        r=[data["kelly"].mean(), data["close"].mean(), sharpe_dict[f"{sym}_kelly"], sharpe_dict[sym]],
        theta=["KellyMean", "CloseMean", "SharpeKelly", "SharpeClose"],
        name=sym,
        fill="toself",
    )
    trace_list.append(trace)
        
fig = go.Figure(trace_list)
fig.update_layout(
    go.Layout(
        polar=dict(radialaxis=dict(visible=True)),
        title=dict({"text": "TVL Top4 Price Kelly Optd Comparision (Radar Chart)", "x": 0.5, "xanchor": "center"}),
    )
)
fig_list_k.append(fig)

eth = kelly_dict["ETH"][["close", "kelly"]]
eth["index"] = np.arange(len(eth))
fig_eth_k = go.Figure(
    ff.create_scatterplotmatrix(
        eth,
        diag="box",
        index="index",
        size=3,
        height=600,
        width=1150,
        colormap="RdBu",
        title={"text": "ETH Price (Scatterplot Matrix)", "x": 0.5, "xanchor": "center"},
    )
)

In [ ]:
for fig in fig_list_k:
    fig.show()
fig_eth_k.show()